In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download alxmamaev/flowers-recognition

In [ ]:
!unzip flowers-recognition.zip

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
from IPython.display import Image, display
from tqdm import tqdm
import cv2
import random as rn
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
listOfImageNames = ['/content/flowers/daisy/100080576_f52e8ee070_n.jpg',
                    '/content/flowers/dandelion/10043234166_e6dd915111_n.jpg',
                    '/content/flowers/rose/10503217854_e66a804309.jpg',
                    '/content/flowers/sunflower/1008566138_6927679c8a.jpg',
                    '/content/flowers/tulip/100930342_92e8746431_n.jpg']
labels = ['daisy','dandelion','rose','sunflower','tulip']
for i,imageName in enumerate(listOfImageNames):
    print(labels[i])
    display(Image(filename=imageName))
    print("\n")

In [ ]:
datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    rotation_range = 40,
    width_shift_range= 0.2,
    height_shift_range= 0.2,
    validation_split=0.2
)

In [ ]:
train_data =  datagen.flow_from_directory(
        "/content/flowers",
        target_size=(150,150),
        batch_size=32,
        subset = 'training',
        class_mode='categorical')

test_data =  datagen.flow_from_directory(
        "/content/flowers",
        target_size=(150,150),
        batch_size=32,
        subset = 'validation',
        class_mode='categorical')

In [ ]:
train_labels = []
traindata = []
for i in range(0,109):
    train_labels.extend(np.array(train_data[i][1]))
    traindata.extend(np.array(train_data[i][0]))
    
test_labels=[]
testdata = []
for i in range(0,27):
    test_labels.extend(np.array(test_data[i][1]))
    testdata.extend(np.array(test_data[i][0]))

In [ ]:
lbls = []
for i in train_labels:
    if all(i == [0.,0.,0.,0.,1.]):
        lbls.append('tulip')
    elif all(i == [0.,0.,0.,1.,0.]):
        lbls.append('sunflower')
    elif all(i == [0.,0.,1.,0.,0.]):
        lbls.append('rose')
    elif all(i == [0.,1.,0.,0.,0.]):
        lbls.append('dendelion')
    elif all(i == [1.,0.,0.,0.,0.]):
        lbls.append('daisy')

In [ ]:
fig,ax=plt.subplots(5,3)
fig.set_size_inches(15,15)
for i in range(5):
    for j in range (3):
        l=rn.randint(0,len(train_labels))
        ax[i,j].imshow(traindata[l])
        ax[i,j].set_title('Flower: '+lbls[l])
        
plt.tight_layout()

In [ ]:
traindata = np.array(traindata)
testdata = np.array(testdata)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [ ]:
traindata.shape


In [ ]:
model = Sequential()


In [ ]:
model.add(Conv2D(filters=32,activation='relu',kernel_size=3,input_shape=[150,150,3]))
model.add(MaxPool2D(pool_size=2,padding='same'))
model.add(Conv2D(filters=64,activation='relu',kernel_size=3))
model.add(MaxPool2D(pool_size=2,padding='same'))
model.add(Conv2D(filters=96,activation='relu',kernel_size=3))
model.add(MaxPool2D(pool_size=2,padding='same'))
model.add(Conv2D(filters=96,activation='relu',kernel_size=3))
model.add(MaxPool2D(pool_size=2,padding='same'))
model.add(Flatten())

model.add(Dense(512,activation='relu'))

model.add(Dense(5,activation='softmax'))

model.summary()

In [ ]:
es = EarlyStopping(monitor='val_loss',patience=3,mode='min')


In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
model.fit(traindata,train_labels,verbose=1,validation_data=(testdata,test_labels),batch_size=128,epochs=5,callbacks=[es])


In [ ]:
losses = model.history.history


In [ ]:
testdata.shape

In [ ]:
pred=model.predict(testdata)
pred_digits=np.argmax(pred,axis=1)

In [ ]:
pred_digits

In [ ]:
print('The predicted class is:', pred_digits[0])


In [ ]:
img_height = 150
img_width = 150
# sunflower_url = "/content/flowers/tulip_test.jpg"
# sunflower_path = tf.keras.utils.get_file('Red_sunflower', origin=sunflower_url)

img_path = "/content/flowers/dandelion.jpg" # Replace with the path to your image file
img = tf.keras.utils.load_img(
    img_path, target_size=(img_height, img_width)
)

img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(labels[np.argmax(score)], 100 * np.max(score))
)